# 表示回路１テストプログラム
- DDR4(VRAM)に格納された画像を表示する
- PLにはDMA(AXIバスマスター側)と表示回路を実装

### [STEP-1] 定数定義
- 表示回路１の解像度はVGA固定
- レジスタは、DISPADDR, DISPCTRL(DISPONビット)のみ有効

In [24]:
# 表示解像度（VGA）
PIXEL_HEIGHT = 480
PIXEL_WIDTH  = 640

# 表示回路レジスタオフセット
RA_DISPADDR = 0x00000000
RA_DISPCTRL = 0x00000004
RA_DISPINT  = 0x00000008
RA_DISPFIFO = 0x0000000c

### [STEP-2] DisplayPort設定用クラス定義
- Zynq内部の専用レジスタの定義とレジスタ設定

In [25]:
import time
from pynq import MMIO
from pynq.lib.video import *

class LiveDisplayPort(DisplayPort):
    
    # DisplayPortレジスタパラメータ
    __DP_REG_BASE_ADDR__                    = 0xfd4a0000
    __DP_REG_ADDR_RANGE__                   = 0x0000cc20
    __RA_DP_MAIN_STREAM_ENABLE__            = 0x00000084
    __RA_V_BLEND_SET_GLOVAL_ALPHA_REG__     = 0x0000a00c
    __RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT__ = 0x0000b070
    __RA_AV_BUF_AUD_VID_CLK_SOURCE__        = 0x0000b120
    __RA_AV_BUF_SRST_REG__                  = 0x0000b124
    
    def __init__(self, event_loop=None):
        super().__init__(event_loop)
        
    def configure(self, mode, pixelformat):
        super().configure(mode, pixelformat)
        self.__setup_live_video__()
        
    def __setup_live_video__(self):
        dpreg = MMIO(self.__DP_REG_BASE_ADDR__, self.__DP_REG_ADDR_RANGE__)
        dpreg.write(self.__RA_DP_MAIN_STREAM_ENABLE__, 0x00000000)
        dpreg.write(self.__RA_V_BLEND_SET_GLOVAL_ALPHA_REG__, 0x00000000)
        dpreg.write(self.__RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT__, 0x00000050)
        dpreg.write(self.__RA_AV_BUF_AUD_VID_CLK_SOURCE__, 0x00000002)
        dpreg.write(self.__RA_AV_BUF_SRST_REG__, 0x00000002)
        time.sleep(1)
        dpreg.write(self.__RA_AV_BUF_SRST_REG__, 0x00000000)
        time.sleep(1)
        dpreg.write(self.__RA_DP_MAIN_STREAM_ENABLE__, 0x00000001)

### [STEP-3] Overlay
- 表示回路１のコンフィギュレーション

In [26]:
from pynq import Overlay
pl = Overlay("design_1.bit")

### [STEP-4] DisplayPort初期化
- Live Videoモードに設定（表示回路の画像を出力するモード）
- 表示回路１では、解像度はVGA固定

In [29]:
lvdp = LiveDisplayPort()
lvdp.configure(VideoMode(PIXEL_WIDTH, PIXEL_HEIGHT, 24), PIXEL_RGB)

### [STEP-5] VRAM領域確保
- ここではまず１フレーム分の領域を確保
- 高さ x 幅 x 4byte(RGBA)で１フレーム

In [30]:
from pynq import allocate

fb0 = allocate(shape = (PIXEL_HEIGHT, PIXEL_WIDTH, 4), dtype = "u1")

### [STEP-6] VRAM領域にテストパターン画像を書き込み
- パターン表示回路で表示した市松模様のパターン
- データの並びは、[B-8bit, G-8bit, R-8bit, A-8bit]の順- ピクセル単位で書き込みをしているため、処理に数分かかります

In [33]:
PIXEL_PATTERN = [[0,0,255,0],[0,255,0,0],[255,0,0,0],[255,0,255,0],[0,255,255,0],[255,255,0,0],[0,0,0,0],[255,255,255,0]]

for h in range(PIXEL_HEIGHT):
    for w in range(PIXEL_WIDTH):
        fb0[h][w][:] = PIXEL_PATTERN[w//(PIXEL_WIDTH>>3)-h//(PIXEL_HEIGHT>>2)]

### [STEP-7] 表示回路のレジスタ設定
- 表示開始先頭アドレス設定 @ DISPADDR
- 表示ON @ DISPCTRL

In [32]:
pl.regbus_0.write(RA_DISPADDR, fb0.device_address)
pl.regbus_0.write(RA_DISPCTRL, 0x00000001)

### [STEP-8] 表示レジスタの設定
- 表示OFF @ DISPCTRL

In [34]:
pl.regbus_0.write(RA_DISPCTRL, 0x00000000)

### [STEP-9] PYNQの初期画面に戻す
- DisplayPortの制御は、PL側からPS側に切り替わります。

In [35]:
lvdp.close()